# Choosing the Best Location for a New Well
In this project, OilyGiant mining company is aiming to build a new well, and they need to determine which of three locations will be in the most profitable. First, the three datasets will be observed and cleaned. Next, linear regression will be run on each dataset. Then, bootstrapping will be used to estimate the most profitable location. 

## Initialization

To begin, I will import libraries needed throughout the project, load the dataset, and observe the data.

In [27]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import numpy as np

### Loading Datasets

In [2]:
# loading datasets 
try:
    geo0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    print("One or more data files could not be read.")

### Check Data

In [3]:
# view data of geo0
geo0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
# observe first few rows of geo0
geo0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
# view data of geo1
geo1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
# observe first few rows of geo1
geo1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
# view data of geo2
geo2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
# observe first few rows of geo2
geo2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Each dataset has the same columns - the unique well identifier (`id`), three unique features, and `product` (volume of reseves in the oil well). All of the datasets have 100,000 rows with no missing values.

### Data Cleaning

The only cleaning we have to do before preparing the data for our model is dropping the `id` column from each dataset. If we attempt to train a dataset while having this unique identifier in the dataset, the model will get confused and may try to make associations about this column.

In [9]:
# drop id columns
geo0 = geo0.drop(['id'], axis=1)
geo1 = geo1.drop(['id'], axis=1)
geo2 = geo2.drop(['id'], axis=1)

In [10]:
# check new data 
geo0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


The `id` column has been dropped.

## Train + Test Model

Linear regression will be used for the model. For each region (geo0, geo1, and geo2), I will split the data into training and validation sets, train each linear regression model on the training sets, and compare the output with our validation set. RSME will be used as our comparison metric.

In [11]:
# method to get features and target of each dataset
def get_features_and_target(data):
    features = data.drop(['product'], axis = 1)
    target = data['product']
    
    return features, target

In [12]:
# method to predict and return values based on our training set
def get_predicted_values_from_regression(features_train, features_valid, target_train):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    return predicted_valid

In [13]:
# method to return rmse given our target and our predicted values
def get_rmse(target_valid, predicted_valid):
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    
    return rmse

Now that I've defined some methods to call, I will loop through the datasets to calculate the predicted mean and the RMSE.

In [14]:
# datasets to loop through
regions = [geo0, geo1, geo2]

# store the valid targets and predictions. each index corresponds with the index in the
# regions dataset
target_valids = []
predicted_valids = []

# loop through datasets, calculate predicted_mean and rmse for each based on linear regression
for i in range(3):
    region = regions[i]
    features, target = get_features_and_target(region)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 12345)
    target_valids.append(target_valid)
    
    predicted_valid = get_predicted_values_from_regression(features_train, features_valid, target_train)
    predicted_valids.append(predicted_valid)
    predicted_mean = predicted_valid.mean()
        
    rmse = get_rmse(target_valid, predicted_valid)
    
    print(f'Region {i}: Mean - {predicted_mean:.2f}, RMSE - {rmse:.2f}')

Region 0: Mean - 92.59, RMSE - 37.58
Region 1: Mean - 68.73, RMSE - 0.89
Region 2: Mean - 94.97, RMSE - 40.03


From the linear regression model, we can see that Regions 0 and 2 had the average volume of predicted reserves (92-94 each). RSME, which we want to be low, displays the difference between our predicted values and actual values. Region 1 had the lowest RMSE in addition to the lowest average prediction, meaning that our Region 1 model was actually the most accurate of our models.

## Profit Calculation

To calculate profit, I will reference the instructions we were given. There are **500 points** in each study, and we need to pick the **best 200** points for our calculation. We want to develop **200 oil wells**, and our total budget is **\$100 million**. One unit of product (1000 barrels) gives us a revenue of **\$4500**.

First, I will calculate the volume of reserves sufficient for developing a new well without losses.

In [15]:
points = 200
total_points = 500
budget = 100000000
revenue_per_unit = 4500
revenue_per_barrel = revenue_per_unit / 1000

# calculate the cost per well - how much to spend per point
cost_per_well = budget / points

# calculate the volume of reserves without losses
volume_reserves_no_loss = cost_per_well / revenue_per_unit

print(f'Volume of Reserves Without Losses: {volume_reserves_no_loss:.2f} ')

Volume of Reserves Without Losses: 111.11 


We end up with 111.11 units (where 1 unit of product is 1000 barrels). The mean reserves we predicted for Region 0, Region 1, and Region 2 were 92.59, 68.73, and 94.97. We need 111.11 units to not incur losses - because the mean reserves we predicted are lower than this threshold, none of our regions are currently incurring losses.

In [16]:
# pick wells with the highest values of predictions
def calculate_profit(target, predicted, points, revenue, budget):
    indices = pd.Series(predicted).reset_index(drop=True).sort_values(ascending=False).index
    best_targets = target.iloc[indices][:points]
    return best_targets.sum() * revenue - budget

In [17]:
# check the profit of each region
print(f'Profit of {points} wells across each region:')
for i in range(3):
    target = pd.Series(target_valids[i])
    predicted = pd.Series(predicted_valids[i])
    region_profit = calculate_profit(target, predicted, points, revenue_per_unit, budget)
    print(f'Region {i}: {region_profit}')

Profit of 200 wells across each region:
Region 0: 33208260.43139851
Region 1: 24150866.966815114
Region 2: 27103499.635998324


Of the 3 regions, it appears that Region 0 is the most profitable.

In [31]:
# calculate profit by bootstrapping
def bootstrap_calculate_profit(target_b, predicted_b, total_points, points, revenue, budget):
    # set random state to get new values each time
    state = np.random.RandomState(12345)
    num_samples = 1000
    values = []
    
    # sample 1000 times 
    for i in range(num_samples):
        target_subsample = target_b.reset_index(drop=True).sample(n = total_points, replace = True, random_state = state)
        predictions_subsample = predicted_b.iloc[target_subsample.index]
        values.append(calculate_profit(target_subsample, predictions_subsample, points, revenue, budget))
    return pd.Series(values)

In [40]:
# check the profit of each region
print(f'Profit of {points} wells across each region (bootstrapped):')
for i in range(3):
    target = pd.Series(target_valids[i])
    predicted = pd.Series(predicted_valids[i])
    values = bootstrap_calculate_profit(target, predicted, total_points, points, revenue_per_unit, budget)
    
    average_profit = values.mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, average_profit, st.sem(values))
    losses = (values < 0).sum() / len(values) * 100
    
    print(f'Region {i}')
    print(f'Average Profit: {average_profit}')
    print(f'95% Confidence Interval: {confidence_interval}')
    print(f'Loss Risk: {losses}%\n')

Profit of 200 wells across each region (bootstrapped):
Region 0
Average Profit: 3961649.8480237117
95% Confidence Interval: (3796203.1514797257, 4127096.5445676977)
Loss Risk: 6.9%

Region 1
Average Profit: 4560451.057866608
95% Confidence Interval: (4431472.486639005, 4689429.62909421)
Loss Risk: 1.5%

Region 2
Average Profit: 4044038.665683568
95% Confidence Interval: (3874457.974712804, 4213619.356654332)
Loss Risk: 7.6%



Bootstrapping allows us to estimate the population characteristics given a sample. In this case, by sampling 1000 times, we were able to determine that Region 1 has the greatest average profit of the three regions. It also has the lowest chance of loss and the narrowest confidence interval.

## Conclusion
In this project, we viewed the datasets of three wells, cleaned the data, and performed linear regression. We use bootstrapping to estimate the population characteristics and ultimately determined that region 1 would be the most profitable region for the new well.